In [5]:
!pip install pyomo

In [11]:
import pandas as pd
import numpy as np

frame = pd.read_csv('optimization_data.csv')
frame.head(10)

,Unnamed: 0,client_id,product,channel,score
0,0,0,credit card,call,0.021752
1,1,0,credit card,sms,0.005114
2,2,0,credit,call,0.923996
3,3,0,credit,sms,0.009079
4,4,1,credit card,call,0.017157
5,5,1,credit card,sms,0.006544
6,6,1,credit,call,0.083950
7,7,1,credit,sms,0.045296
8,8,2,credit card,call,0.018123
9,9,2,credit card,sms,0.005872


In [12]:
frame = frame.drop(['Unnamed: 0'], axis=1)

In [13]:
from pyomo.environ import ConcreteModel, Var, quicksum, Objective, Constraint, SolverFactory, Binary, maximize, summation, sum_product, ConstraintList

def optimize(frame: pd.DataFrame, channel_limits: dict) -> list:
    df = frame.copy()

    model = ConcreteModel()

    model.x = Var(range(df.shape[0]), domain=Binary)

    model.objective = Objective(expr=sum_product(df['score'], model.x), sense=maximize)

    model.constraints = ConstraintList()
    for channel, limit in channel_limits.items():
        indices = df[df['channel'] == channel].index.tolist()
        model.constraints.add(quicksum(model.x[i] for i in indices) <= limit)

    for client_id in df['client_id'].unique():
        indices = df[df['client_id'] == client_id].index.tolist()
        model.constraints.add(quicksum(model.x[i] for i in indices) <= 1)
        
    solver = SolverFactory('glpk') # glpk, cbc
    result = solver.solve(model, tee=True)

    return [model.x[i].value for i in range(df.shape[0])]

In [14]:
channel_limits = {'call': 4000, 'sms': 7000}

results_1 = optimize(frame, channel_limits)
frame['optimal_decision_1'] = results_1

#распределение продуктов в каналах
frame[frame['optimal_decision_1'] == 1].groupby(['channel', 'product']).\
                                    agg({'client_id': 'count'}).\
                                    rename(columns={'client_id': 'client_cnt'})

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\kopan\AppData\Local\Temp\tmp7j6_q47f.glpk.raw --wglp C:\Users\kopan\AppData\Local\Temp\tmpr95ef97c.glpk.glp
 --cpxlp C:\Users\kopan\AppData\Local\Temp\tmp4x6e8hkk.pyomo.lp
Reading problem data from 'C:\Users\kopan\AppData\Local\Temp\tmp4x6e8hkk.pyomo.lp'...
C:\Users\kopan\AppData\Local\Temp\tmp4x6e8hkk.pyomo.lp:379924: warning: lower bound of variable 'x2' redefined
C:\Users\kopan\AppData\Local\Temp\tmp4x6e8hkk.pyomo.lp:379924: warning: upper bound of variable 'x2' redefined
20002 rows, 80000 columns, 160000 non-zeros
80000 integer variables, all of which are binary
459924 lines were read
Writing problem data to 'C:\Users\kopan\AppData\Local\Temp\tmpr95ef97c.glpk.glp'...
359915 lines were written
GLPK Integer Optimizer 5.0
20002 rows, 80000 columns, 160000 non-zeros
80000 integer variables, all of which are binary
Preprocessing...
20002 rows, 80000 columns, 160000 non-zeros
80000 integer variab

client_cnt
channel product                
call    credit             1641
        credit card        2359
sms     credit             3635
        credit card        3365

# Оптимизация маржи

In [15]:
revenue_credit_card = 13000
revenue_credit = 10000
cost_call = 50
cost_sms = 1.5

In [16]:
scores = list(frame['score'])
margin_scores = [0 for i in range(len(scores))]

for i in range(len(scores)):
    if i % 4 == 0:
        margin_scores[i] = scores[i] * (revenue_credit_card - cost_call)
    elif i % 4 == 1:
        margin_scores[i] = scores[i] * (revenue_credit_card - cost_sms)
    elif i % 4 == 2:
        margin_scores[i] = scores[i] * (revenue_credit - cost_call)
    else:
        margin_scores[i] = scores[i] * (revenue_credit - cost_sms)

frame['margin_score'] = margin_scores

In [17]:
def optimize_margin(frame: pd.DataFrame, channel_limits: dict) -> list:
    df = frame.copy()
    
    model = ConcreteModel()
    
    model.x = Var(range(df.shape[0]), domain=Binary)
    
    model.objective = Objective(expr=sum_product(df['margin_score'], model.x), sense=maximize)
    
    model.constraints = ConstraintList()
    for channel, limit in channel_limits.items():
        indices = df[df['channel'] == channel].index.tolist()
        model.constraints.add(quicksum(model.x[i] for i in indices) <= limit)

    for client_id in df['client_id'].unique():
        indices = df[df['client_id'] == client_id].index.tolist()
        model.constraints.add(quicksum(model.x[i] for i in indices) <= 1)
        
    solver = SolverFactory('glpk') # glpk, cbc
    result = solver.solve(model, tee=True)

    return [model.x[i].value for i in range(df.shape[0])]

In [18]:
channel_limits = {'call': 4000, 'sms': 7000}

results_2 = optimize_margin(frame, channel_limits)
frame['optimal_decision_2'] = results_2

#распределение продуктов в каналах
frame[frame['optimal_decision_2'] == 1].groupby(['channel', 'product']).\
                                    agg({'client_id': 'count'}).\
                                    rename(columns={'client_id': 'client_cnt'})

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\kopan\AppData\Local\Temp\tmp9ykdkuc2.glpk.raw --wglp C:\Users\kopan\AppData\Local\Temp\tmpd2z9fm_f.glpk.glp
 --cpxlp C:\Users\kopan\AppData\Local\Temp\tmp5av336v5.pyomo.lp
Reading problem data from 'C:\Users\kopan\AppData\Local\Temp\tmp5av336v5.pyomo.lp'...
C:\Users\kopan\AppData\Local\Temp\tmp5av336v5.pyomo.lp:379924: warning: lower bound of variable 'x2' redefined
C:\Users\kopan\AppData\Local\Temp\tmp5av336v5.pyomo.lp:379924: warning: upper bound of variable 'x2' redefined
20002 rows, 80000 columns, 160000 non-zeros
80000 integer variables, all of which are binary
459924 lines were read
Writing problem data to 'C:\Users\kopan\AppData\Local\Temp\tmpd2z9fm_f.glpk.glp'...
359915 lines were written
GLPK Integer Optimizer 5.0
20002 rows, 80000 columns, 160000 non-zeros
80000 integer variables, all of which are binary
Preprocessing...
20002 rows, 80000 columns, 160000 non-zeros
80000 integer variab

client_cnt
channel product                
call    credit             1489
        credit card        2511
sms     credit             3176
        credit card        3824

# Сравнение

Количество общих результатов для двух решений:

In [28]:
len(frame.loc[frame['optimal_decision_1'] == frame['optimal_decision_2']])

78730

Количество общих *сделанных* предложений для двух решений:

In [30]:
sum(np.array(results_1) * np.array(results_2))

10365.0

Количество результатов, которые отличаются для двух решений: 

In [29]:
len(frame.loc[frame['optimal_decision_1'] != frame['optimal_decision_2']])

1270

Разница в количестве предложенных кредитов и кредитных карт в двух решениях при использовании *звонков*: $$\Delta_{call} = 2511 - 2359 = 152$$

Разница в количестве предложенных кредитов и кредитных карт в двух решениях при использовании *SMS*: $$\Delta_{SMS} = 3824 - 3365 = 459$$

**Вывод**. При оптимизации маржи при использовании канала связи SMS есть перекос в сторону кредитных карт, при решении обычной задачи -- наоборот, в сторону кредитов. При использовании звонков в обоих задачах количество кредитных карт получается больше. Если сравнивать два решения, то большинство предложений -- общие и в первой, и во второй задаче, но есть и отличающиеся результаты.